# Tutorial [link](https://github.com/tqdm/tqdm)

In [1]:
from tqdm import tqdm
from time import sleep

# tqdm



``tqdm`` derives from the Arabic word *taqaddum* (تقدّم) which can mean "progress,"
and is an abbreviation for "I love you so much" in Spanish (*te quiero demasiado*).

Instantly make your loops show a smart progress meter - just wrap any
iterable with ``tqdm(iterable)``, and you're done!

```python

    from tqdm import tqdm
    for i in tqdm(range(10000)):
        ...
```
``76%|████████████████████████        | 7568/10000 [00:33<00:10, 229.00it/s]``

``trange(N)`` can be also used as a convenient shortcut for
``tqdm(range(N))``.

It can also be executed as a module with pipes:

```sh

    $ seq 9999999 | tqdm --bytes | wc -l
    75.2MB [00:00, 217MB/s]
    9999999
```
```sh
    $ tar -zcf - docs/ | tqdm --bytes --total `du -sb docs/ | cut -f1` \
        > backup.tgz
     32%|██████████▍                      | 8.89G/27.9G [00:42<01:31, 223MB/s]
```

## Installation

### Latest PyPI stable release

```sh
    pip install tqdm
```


### Latest Conda release

```sh
    conda install -c conda-forge tqdm
```

## Usage

``tqdm`` is very versatile and can be used in a number of ways.
The three main ones are given below.

### Iterable-based
Wrap ``tqdm()`` around any iterable:

In [2]:
from tqdm import tqdm
from time import sleep

text = ""
for char in tqdm(["a", "b", "c", "d"]):
    sleep(0.25)
    text = text + char

100%|██████████| 4/4 [00:01<00:00,  3.95it/s]


``trange(i)`` is a special optimised instance of ``tqdm(range(i))``:

In [3]:
from tqdm import trange

for i in trange(100):
    sleep(0.01)

100%|██████████| 100/100 [00:01<00:00, 95.81it/s]


Instantiation outside of the loop allows for manual control over ``tqdm()``:

In [4]:
pbar = tqdm(["a", "b", "c", "d"])
for char in pbar:
    sleep(0.25)
    pbar.set_description("Processing %s" % char)

Processing d: 100%|██████████| 4/4 [00:01<00:00,  3.91it/s]


## Manual

Manual control of ``tqdm()`` updates using a ``with`` statement:

In [5]:
with tqdm(total=100) as pbar:
    for i in range(10):
        sleep(0.1)
        pbar.update(10)

100%|██████████| 100/100 [00:01<00:00, 97.64it/s]


If the optional variable ``total`` (or an iterable with ``len()``) is
provided, predictive stats are displayed.

``with`` is also optional (you can just assign ``tqdm()`` to a variable,
but in this case don't forget to ``del`` or ``close()`` at the end:

In [6]:
pbar = tqdm(total=100)
for i in range(10):
    sleep(0.1)
    pbar.update(10)
pbar.close()

100%|██████████| 100/100 [00:01<00:00, 97.59it/s]


## Module

Perhaps the most wonderful use of ``tqdm`` is in a script or on the command
line. Simply inserting ``tqdm`` (or ``python -m tqdm``) between pipes will pass
through all ``stdin`` to ``stdout`` while printing progress to ``stderr``.

The example below demonstrate counting the number of lines in all Python files
in the current directory, with timing information included.

```sh
    $ time find . -name '*.py' -type f -exec cat \{} \; | wc -l
    857365

    real    0m3.458s
    user    0m0.274s
    sys     0m3.325s

    $ time find . -name '*.py' -type f -exec cat \{} \; | tqdm | wc -l
    857366it [00:03, 246471.31it/s]
    857365

    real    0m3.585s
    user    0m0.862s
    sys     0m3.358s
```

Note that the usual arguments for ``tqdm`` can also be specified.
```sh
    $ find . -name '*.py' -type f -exec cat \{} \; |
        tqdm --unit loc --unit_scale --total 857366 >> /dev/null
    100%|█████████████████████████████████| 857K/857K [00:04<00:00, 246Kloc/s]
```

Backing up a large directory?

```sh
    $ tar -zcf - docs/ | tqdm --bytes --total `du -sb docs/ | cut -f1` \
      > backup.tgz
     44%|██████████████▊                   | 153M/352M [00:14<00:18, 11.0MB/s]
```

This can be beautified further:

```sh
    $ BYTES="$(du -sb docs/ | cut -f1)"
    $ tar -cf - docs/ \
      | tqdm --bytes --total "$BYTES" --desc Processing | gzip \
      | tqdm --bytes --total "$BYTES" --desc Compressed --position 1 \
      > ~/backup.tgz
    Processing: 100%|██████████████████████| 352M/352M [00:14<00:00, 30.2MB/s]
    Compressed:  42%|█████████▎            | 148M/352M [00:14<00:19, 10.9MB/s]
```

Or done on a file level using 7-zip:

```sh
    $ 7z a -bd -r backup.7z docs/ | grep Compressing \
      | tqdm --total $(find docs/ -type f | wc -l) --unit files \
      | grep -v Compressing
    100%|██████████████████████████▉| 15327/15327 [01:00<00:00, 712.96files/s]
```

Pre-existing CLI programs already outputting basic progress information will
benefit from ``tqdm``'s ``--update`` and ``--update_to`` flags:

```sh
    $ seq 3 0.1 5 | tqdm --total 5 --update_to --null
    100%|████████████████████████████████████| 5.0/5 [00:00<00:00, 9673.21it/s]
    $ seq 10 | tqdm --update --null  # 1 + 2 + ... + 10 = 55 iterations
    55it [00:00, 90006.52it/s]
```

## Documentation
```python
    class tqdm():
      """
      Decorate an iterable object, returning an iterator which acts exactly
      like the original iterable, but prints a dynamically updating
      progressbar every time a value is requested.
      """

      def __init__(self, iterable=None, desc=None, total=None, leave=True,
                   file=None, ncols=None, mininterval=0.1,
                   maxinterval=10.0, miniters=None, ascii=None, disable=False,
                   unit='it', unit_scale=False, dynamic_ncols=False,
                   smoothing=0.3, bar_format=None, initial=0, position=None,
                   postfix=None, unit_divisor=1000):
```

### Parameters
* iterable  : iterable, optional  
    Iterable to decorate with a progressbar.
    Leave blank to manually manage the updates.
* desc  : str, optional  
    Prefix for the progressbar.
* total  : int or float, optional  
    The number of expected iterations. If unspecified,
    len(iterable) is used if possible. If float("inf") or as a last
    resort, only basic progress statistics are displayed
    (no ETA, no progressbar).
    If ``gui`` is True and this parameter needs subsequent updating,
    specify an initial arbitrary large positive number,
    e.g. 9e9.
* leave  : bool, optional  
    If [default: True], keeps all traces of the progressbar
    upon termination of iteration.
    If ``None``, will leave only if ``position`` is ``0``.
* file  : ``io.TextIOWrapper`` or ``io.StringIO``, optional  
    Specifies where to output the progress messages
    (default: sys.stderr). Uses ``file.write(str)`` and ``file.flush()``
    methods.  For encoding, see ``write_bytes``.
* ncols  : int, optional  
    The width of the entire output message. If specified,
    dynamically resizes the progressbar to stay within this bound.
    If unspecified, attempts to use environment width. The
    fallback is a meter width of 10 and no limit for the counter and
    statistics. If 0, will not print any meter (only stats).
* mininterval  : float, optional  
    Minimum progress display update interval [default: 0.1] seconds.
* maxinterval  : float, optional  
    Maximum progress display update interval [default: 10] seconds.
    Automatically adjusts ``miniters`` to correspond to ``mininterval``
    after long display update lag. Only works if ``dynamic_miniters``
    or monitor thread is enabled.
* miniters  : int or float, optional  
    Minimum progress display update interval, in iterations.
    If 0 and ``dynamic_miniters``, will automatically adjust to equal
    ``mininterval`` (more CPU efficient, good for tight loops).
    If > 0, will skip display of specified number of iterations.
    Tweak this and ``mininterval`` to get very efficient loops.
    If your progress is erratic with both fast and slow iterations
    (network, skipping items, etc) you should set miniters=1.
* ascii  : bool or str, optional  
    If unspecified or False, use unicode (smooth blocks) to fill
    the meter. The fallback is to use ASCII characters " 123456789#".
* disable  : bool, optional  
    Whether to disable the entire progressbar wrapper
    [default: False]. If set to None, disable on non-TTY.
* unit  : str, optional  
    String that will be used to define the unit of each iteration
    [default: it].
* unit_scale  : bool or int or float, optional  
    If 1 or True, the number of iterations will be reduced/scaled
    automatically and a metric prefix following the
    International System of Units standard will be added
    (kilo, mega, etc.) [default: False]. If any other non-zero
    number, will scale ``total`` and ``n``.
* dynamic_ncols  : bool, optional  
    If set, constantly alters ``ncols`` and ``nrows`` to the
    environment (allowing for window resizes) [default: False].
* smoothing  : float, optional  
    Exponential moving average smoothing factor for speed estimates
    (ignored in GUI mode). Ranges from 0 (average speed) to 1
    (current/instantaneous speed) [default: 0.3].
* bar_format  : str, optional  
    Specify a custom bar string formatting. May impact performance.
    [default: '{l_bar}{bar}{r_bar}'], where
    l_bar='{desc}: {percentage:3.0f}%|' and
    r_bar='| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, '
    '{rate_fmt}{postfix}]'
    Possible vars: l_bar, bar, r_bar, n, n_fmt, total, total_fmt,
    percentage, elapsed, elapsed_s, ncols, nrows, desc, unit,
    rate, rate_fmt, rate_noinv, rate_noinv_fmt,
    rate_inv, rate_inv_fmt, postfix, unit_divisor,
    remaining, remaining_s, eta.
    Note that a trailing ": " is automatically removed after {desc}
    if the latter is empty.
* initial  : int or float, optional  
    The initial counter value. Useful when restarting a progress
    bar [default: 0]. If using float, consider specifying ``{n:.3f}``
    or similar in ``bar_format``, or specifying ``unit_scale``.
* position  : int, optional  
    Specify the line offset to print this bar (starting from 0)
    Automatic if unspecified.
    Useful to manage multiple bars at once (eg, from threads).
* postfix  : dict or ``*``, optional  
    Specify additional stats to display at the end of the bar.
    Calls ``set_postfix(**postfix)`` if possible (dict).
* unit_divisor  : float, optional  
    [default: 1000], ignored unless ``unit_scale`` is True.
* write_bytes  : bool, optional  
    If (default: None) and ``file`` is unspecified,
    bytes will be written in Python 2. If ``True`` will also write
    bytes. In all other cases will default to unicode.
* lock_args  : tuple, optional  
    Passed to ``refresh`` for intermediate output
    (initialisation, iterating, and updating).
* nrows  : int, optional  
    The screen height. If specified, hides nested bars outside this
    bound. If unspecified, attempts to use environment height.
    The fallback is 20.
* colour  : str, optional  
    Bar colour (e.g. 'green', '#00ff00').
* delay  : float, optional  
    Don't display until [default: 0] seconds have elapsed.

### Extra CLI Options
* delim  : chr, optional  
    Delimiting character [default: '\n']. Use '\0' for null.
    N.B.: on Windows systems, Python converts '\n' to '\r\n'.
* buf_size  : int, optional  
    String buffer size in bytes [default: 256]
    used when ``delim`` is specified.
* bytes  : bool, optional  
    If true, will count bytes, ignore ``delim``, and default
    ``unit_scale`` to True, ``unit_divisor`` to 1024, and ``unit`` to 'B'.
* tee  : bool, optional  
    If true, passes ``stdin`` to both ``stderr`` and ``stdout``.
* update  : bool, optional  
    If true, will treat input as newly elapsed iterations,
    i.e. numbers to pass to ``update()``. Note that this is slow
    (~2e5 it/s) since every input must be decoded as a number.
* update_to  : bool, optional  
    If true, will treat input as total elapsed iterations,
    i.e. numbers to assign to ``self.n``. Note that this is slow
    (~2e5 it/s) since every input must be decoded as a number.
* null  : bool, optional  
    If true, will discard input (no stdout).
* manpath  : str, optional  
    Directory in which to install tqdm man pages.
* comppath  : str, optional  
    Directory in which to place tqdm completion.
* log  : str, optional  
    CRITICAL|FATAL|ERROR|WARN(ING)|[default: 'INFO']|DEBUG|NOTSET.

### Returns
* out  : decorated iterator.  

```python
    class tqdm():
      def update(self, n=1):
          """
          Manually update the progress bar, useful for streams
          such as reading files.
          E.g.:
          >>> t = tqdm(total=filesize) # Initialise
          >>> for current_buffer in stream:
          ...    ...
          ...    t.update(len(current_buffer))
          >>> t.close()
          The last line is highly recommended, but possibly not necessary if
          ``t.update()`` will be called in such a way that ``filesize`` will be
          exactly reached and printed.

          Parameters
          ----------
          n  : int or float, optional
              Increment to add to the internal counter of iterations
              [default: 1]. If using float, consider specifying ``{n:.3f}``
              or similar in ``bar_format``, or specifying ``unit_scale``.

          Returns
          -------
          out  : bool or None
              True if a ``display()`` was triggered.
          """

      def close(self):
          """Cleanup and (if leave=False) close the progressbar."""

      def clear(self, nomove=False):
          """Clear current bar display."""

      def refresh(self):
          """
          Force refresh the display of this bar.

          Parameters
          ----------
          nolock  : bool, optional
              If ``True``, does not lock.
              If [default: ``False``]: calls ``acquire()`` on internal lock.
          lock_args  : tuple, optional
              Passed to internal lock's ``acquire()``.
              If specified, will only ``display()`` if ``acquire()`` returns ``True``.
          """

      def unpause(self):
          """Restart tqdm timer from last print time."""

      def reset(self, total=None):
          """
          Resets to 0 iterations for repeated use.

          Consider combining with ``leave=True``.

          Parameters
          ----------
          total  : int or float, optional. Total to use for the new bar.
          """

      def set_description(self, desc=None, refresh=True):
          """
          Set/modify description of the progress bar.

          Parameters
          ----------
          desc  : str, optional
          refresh  : bool, optional
              Forces refresh [default: True].
          """

      def set_postfix(self, ordered_dict=None, refresh=True, **tqdm_kwargs):
          """
          Set/modify postfix (additional stats)
          with automatic formatting based on datatype.

          Parameters
          ----------
          ordered_dict  : dict or OrderedDict, optional
          refresh  : bool, optional
              Forces refresh [default: True].
          kwargs  : dict, optional
          """

      @classmethod
      def write(cls, s, file=sys.stdout, end="\n"):
          """Print a message via tqdm (without overlap with bars)."""

      @property
      def format_dict(self):
          """Public API for read-only member access."""

      def display(self, msg=None, pos=None):
          """
          Use ``self.sp`` to display ``msg`` in the specified ``pos``.

          Consider overloading this function when inheriting to use e.g.:
          ``self.some_frontend(**self.format_dict)`` instead of ``self.sp``.

          Parameters
          ----------
          msg  : str, optional. What to display (default: ``repr(self)``).
          pos  : int, optional. Position to ``moveto``
            (default: ``abs(self.pos)``).
          """

      @classmethod
      @contextmanager
      def wrapattr(cls, stream, method, total=None, bytes=True, **tqdm_kwargs):
          """
          stream  : file-like object.
          method  : str, "read" or "write". The result of ``read()`` and
              the first argument of ``write()`` should have a ``len()``.

          >>> with tqdm.wrapattr(file_obj, "read", total=file_obj.size) as fobj:
          ...     while True:
          ...         chunk = fobj.read(chunk_size)
          ...         if not chunk:
          ...             break
          """

      @classmethod
      def pandas(cls, *targs, **tqdm_kwargs):
          """Registers the current `tqdm` class with `pandas`."""

    def trange(*args, **tqdm_kwargs):
        """
        A shortcut for `tqdm(xrange(*args), **tqdm_kwargs)`.
        On Python3+, `range` is used instead of `xrange`.
        """
```

### Pandas Integration

Due to popular demand we've added support for ``pandas`` -- here's an example
for ``DataFrame.progress_apply`` and ``DataFrameGroupBy.progress_apply``:

In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm

df = pd.DataFrame(np.random.randint(0, 100, (100000, 6)))

# Register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
# (can use `tqdm.gui.tqdm`, `tqdm.notebook.tqdm`, optional kwargs, etc.)
tqdm.pandas(desc="my bar!")

# Now you can use `progress_apply` instead of `apply`
# and `progress_map` instead of `map`
df.progress_apply(lambda x: x**2)
# can also groupby:
# df.groupby(0).progress_apply(lambda x: x**2)

my bar!: 100%|██████████| 6/6 [00:00<00:00, 832.51it/s]


,0,1,2,3,4,5
0,9409,1,6084,8836,2809,6561
1,9604,841,576,5625,36,6084
2,961,3721,1764,5329,1369,7744
3,2025,4096,1444,2209,8281,900
4,1521,8464,576,1089,6241,4489
...,...,...,...,...,...,...
99995,7396,729,3249,900,1764,196
99996,2809,1521,5184,7569,441,4356
99997,9216,6084,169,529,6400,3364
99998,9,16,2500,9,3844,4624


### IPython/Jupyter Integration

IPython/Jupyter is supported via the ``tqdm.notebook`` submodule:

```python
    from tqdm.notebook import trange, tqdm
    from time import sleep

    for i in trange(3, desc='1st loop'):
        for j in tqdm(range(100), desc='2nd loop'):
            sleep(0.01)
```
In addition to ``tqdm`` features, the submodule provides a native Jupyter
widget (compatible with IPython v1-v4 and Jupyter), fully working nested bars
and colour hints (blue: normal, green: completed, red: error/interrupt,
light blue: no ETA); as demonstrated below.


The ``notebook`` version supports percentage or pixels for overall width
(e.g.: ``ncols='100%'`` or ``ncols='480px'``).

It is also possible to let ``tqdm`` automatically choose between
console or notebook versions by using the ``autonotebook`` submodule:

```python
    from tqdm.autonotebook import tqdm
    tqdm.pandas()
```
Note that this will issue a ``TqdmExperimentalWarning`` if run in a notebook
since it is not meant to be possible to distinguish between ``jupyter notebook``
and ``jupyter console``. Use ``auto`` instead of ``autonotebook`` to suppress
this warning.

Note that notebooks will display the bar in the cell where it was created.
This may be a different cell from the one where it is used.
If this is not desired, either

- delay the creation of the bar to the cell where it must be displayed, or
- create the bar with ``display=False``, and in a later cell call
  ``display(bar.container)``:

```python
    from tqdm.notebook import tqdm
    pbar = tqdm(..., display=False)


    # different cell
    display(pbar.container)
```